# 事前アンケートの質問項目

0. タイムスタンプ
1. 【Q1】What is your age ?
2. 【Q2】What gender do you identify as ?
3. 【Q3】Which geographical region are you currently working or attending school/university in ?
4. 【Q4】Which geographical region do you most strongly associate with ?
5. 【Q5】What is your job title ?
6. 【Q6】Which group do you belong to ?
7. 【Q7】What is your research type ?
8. 【Q8】How long have you been in this field ?
9. 【Q9】Are you satisfied with your career to date ?
10. 【Q10】How many hours, on average, do you spend on housework, childcare, and caregiving per day ?
11. 【Q11】Did you already sign up for the diversity session in ICRC2023?
12. 【Q12】What do you think about the initiatives on DE&I of your group? [Gender balance]
13. 【Q12】What do you think about the initiatives on DE&I of your group? [Diversity]
14. 【Q12】What do you think about the initiatives on DE&I of your group? [Equity]
15. 【Q12】What do you think about the initiatives on DE&I of your group? [Inclusion]
16. 【Q13】What is the percentage of female researcher in your group?
17. 【Q14】What do you think about the percentage above ?
18. 【Q15】Please let us know If your group has any good practice examples related to DE&I ?
19. 【Q16】Please let us know if there is anything your group needs to work on or if your group has any problems related to DE&I.
20. 【Q17】What are your thoughts on diversity, equity & inclusion initiatives ? [Gender balance]
21. 【Q17】What are your thoughts on diversity, equity & inclusion initiatives ? [Diversity]
22. 【Q17】What are your thoughts on diversity, equity & inclusion initiatives ? [Equity]
23. 【Q17】What are your thoughts on diversity, equity & inclusion initiatives ? [Inclusion]
24. 【Q18】Could you tell us more about your thoughts (agree / disagree) ?
25. 【Q19】When did you first become interested in science ?
26. 【Q20】Do you have any concerns / problems related to DE&I initiatives in science ?
27. 【Q21】What reasons do you think are hindering DE&I initiatives in science ?
28. 【Q22】Comments

Pythonのライブラリ
- ``pathlib.Path`` : パス操作
- ``pandas`` : データ集計
- ``altair (v5)`` : プロット作成

In [216]:
from pathlib import Path
import pandas as pd
import altair as alt

print(f"Pandas : {pd.__version__}")
print(f"Altair : {alt.__version__}")

Pandas : 2.0.3
Altair : 5.0.1


事前アンケートの結果を読み込む

- 回答データはGoogleスプレッドシートからCSV形式でダウンロード（手動）
- ファイルパスは``../data/test_data/``にする（あとで変えるかも）
- 回答時刻（``timestamp``カラム）はDateTimeオブジェクトに変換する
- 回答数を集計（主に``sum``）するためのカラム``response``を追加する

In [217]:
fname = Path("../data/test_data/20230710_icrc2023_diversity_presurvey_answers.csv")
data = pd.read_csv(fname, skiprows=1, parse_dates=["timestamp"])
data["response"] = 1

In [221]:
# データフレームを確認する（不要なときはコメントアウトしておく）
# data.head()
# data.columns

回答時刻を調べてみる

- ``timestamp``カラムを使って、回答時刻の分布を調べる
- 記録されている日時は、たぶんUTC+0900？（よく分かってない）
- 年月日（=date）ごとにリサンプルして合計したいので、データフレームのインデックスを``timestamp``にする
- ``date``と``時刻``のヒートマップを作りたいので、それ用のカラムを追加する

In [223]:
data.index = data["timestamp"]
tm = data.resample("H")["response"].sum().reset_index()
# tm["yyyymmdd"] = tm["timestamp"].dt.date
tm["HH"] = tm["timestamp"].dt.hour
# tm.info()

In [225]:
tm

,timestamp,response,HH
0,2023-07-08 08:00:00,4,8
1,2023-07-08 09:00:00,15,9
2,2023-07-08 10:00:00,6,10
3,2023-07-08 11:00:00,5,11
4,2023-07-08 12:00:00,3,12
...,...,...,...
69,2023-07-11 05:00:00,1,5
70,2023-07-11 06:00:00,0,6
71,2023-07-11 07:00:00,2,7
72,2023-07-11 08:00:00,0,8


回答

In [226]:
alt.Chart(data_time).mark_rect().encode(
    alt.Y("date(timestamp):T").title("記入日"),
    alt.X("HH:O").title("時刻"),
    alt.Color("response"),
).properties(
    width=800,
    height=400,
).save("trend.png")

すべての質問に対してヒストグラムを作成する

- 横軸を「質問番号」にしたヒストグラムを作成する
- 横軸はカテゴリカルなデータにして、任意の順番に並べたい（TODO）
  - いまは自動でアルファベット順に整列

In [227]:
def make_histogram(data: pd.DataFrame, x: str):
    grouped = data.groupby(x)["response"].sum().reset_index().sort_values(by=x)

    chart = alt.Chart(grouped).mark_bar().encode(
        alt.X(x),
        alt.Y("response"),
    ).properties(
        width=500,
        height=500,
    )
    return {"data": grouped, "chart": chart}

In [229]:
q = make_histogram(data, "q2")
q["data"]
q["chart"]

alt.Chart(...)

In [230]:
# すべてのカラムでループ
# ただし timestampとresponseは除外
for c in data.columns[1:-1]:    
    hist = make_histogram(data, c)
    fname = f"../data/quick_summary/{c}.csv"
    hist["data"].to_csv(fname, index=False)
    fname = f"../data/quick_summary/{c}.png"
    hist["chart"].save(fname)    
    

2つの質問をクロス集計する

- すべてのカラムに対してクロス集計する
- CSV集計するときの見出し列をわかりやすくするために、転置（``.T``）している
  - もっとコードが読みやすくなる方法があるかもしれない

In [257]:
ctabs = {}
for c1 in data.columns[1:-1]:
    for c2 in data.columns[1:-1]:
        ctab = pd.crosstab(data[c1], data[c2])
        name = f"{c1}_{c2}"
        fname = f"../data/quick_summary/tmp_crosstab_{name}.csv"
        ctab.T.to_csv(fname)
        ctabs.update({name: ctab})